# Homework 2 Continuing to Explore Graph Edge Data
---
Like Homework 2, below you will find a number of questions with space to place and run your code.

Also like Homework 2, several qeustions build upon previous question results, so prgress in a sequential manner, and verifying your results befor proceeding to the next question.

---

In [5]:
import csv
import math
from io import StringIO
from functools import partial
from typing import Dict, Tuple, List
from sortedcontainers import SortedKeyList

In [6]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("spark://127.0.0.1:7077")
    # the number of executors this job needs
    .config("spark.executor.instances", 2)
    # the number of CPU cores memory this needs from the executor,
    # it would be reserved on the worker
    .config("spark.executor.cores", "2")
    .config("spark.executor.memory", "4G")
    .getOrCreate()
)
sc = spark.sparkContext

Py4JJavaError: An error occurred while calling o36.applyModifiableSettings.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:98)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:81)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:144)
	at org.apache.spark.sql.SparkSession$$Lambda$1117/765451161.apply(Unknown Source)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:144)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:143)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:163)
	at org.apache.spark.sql.SparkSession$$Lambda$1116/891629061.apply(Unknown Source)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.

## Question 1: Reading a CSV file and constructing an RDD (5 points)
---
In this question, like in Homework 2, you will read the `edges_table.csv` file that was included with this assignment. However, this time we will be using the built in text file reading ability of the SparkContext object to read our file.

Recall the text file reading will either read each line of the file and return them as entries in an RDD or will read the entire file and return it's name and the file contents as a pair entry in an RDD. Ideally we would like to use the line by line, but our data has a header line in the file which we don't want included in our dataset. So, you will need to use the SparkContext method that will read the whole file, then map the file that was read into a function that will parse the file removing the header, and then placing each of the lines into a resulting RDD with a single record per line.  

__NOTE:__ All of the worker processes have to see the file in the same location that you pass in as the file location. If you pass in a relative location to your driver program, I believe that Spark will attach an ablsolute path location to that relative path by adding the path to where your driver is running. This would be useful in case your worker process isn't started from the same directory as your driver program, but I have not confirmed this to be the case, so you may need to play around with the file path information you pass in.


In [ ]:
#Place your answer here.
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

edges_table_rdd = sc.textFile("edges_table.csv")

edges_header = edges_table_rdd.first()

edges_no_header = edges_table_rdd.filter(lambda line: line != edges_header)

edges_result = edges_no_header.map(lambda line: line.split(","))

Py4JJavaError: An error occurred while calling o31.defaultParallelism.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:408)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.Cli

## Question 2: Constructing Vertex Data Structure RDDs (5 points)
---
The data that was loaded in Question 1 is just a table of directed edges in some graph.  The first entry of a row is the `from` vertex and the second is the `to` vertex.  

What you should do in this question is create an RDD that contains a key/value pair where the key is the `from vertex number` and the value should be a dictionary that contins:
  * A list of vertices that the vertex links to named `out_links`
  * A list of vertices that link to the vertex indicated by the key called `in_links`

Your Homework 2 should already have this answer for you.

__NOTE:__ Persist this RDD as it will be used multiple times in later questions.

---

In [ ]:
#Place your answer here.
pair_rdd = edges_result.map(lambda x: (x[0], x[1]))

out_links = pair_rdd.groupByKey().mapValues(list)
in_links = pair_rdd.map(lambda x: (x[1], x[0])).groupByKey().mapValues(list)

vertex_structure_rdd = out_links.leftOuterJoin(in_links).mapValues(lambda x: {'out_links': x[0], 'in_links': x[1] if x[1] else []})

vertex_structure_rdd.persist()

NameError: name 'edges_result' is not defined

## Question 3: Write a function to process join tuples (10 points)
---
In a later question we will be repeatedly joining our result from the previous question with another RDD that contains information we will be updating in an iterative algorithm. This join will be so that we can use the information in both RDDs to update the information in the new RDD. So, in order to do this update process, lets write a function that will process one entry of that join result.

Each line in the RDD we will be processing will contain a Tuple with the following format:
 * Item[0] is an integer indicating which vertex we are processing data for
 * Item[1] is another Tuple with a Dict in position 0 that is our vertex link information and a float in position 1 that is the current rank of the vertex being processed.

We don't care about the current vertex ID so we will ignore this value in this function. We do, however, care about both the rank and the list of vertices this vertex is linked to.  

For the output of this function you should return a list of key/value tuples. The key should be the vertex that this vertex has an edge to. The value should be the current rank times `1/N` where `N` is the count of edges going out from this vertex.

---

In [ ]:
#Place your answer here.
def rank_order(entry):
    vertex_id = entry[0]
    data = entry[1]
    vertex_info = data[0]
    current_rank = data[1]
    out_link = vertex_info['out_link']
    num_out_links = len(out_link)
    rank_order = current_rank / num_out_links if num_out_links > 0 else 0
    return [(out_vertex, rank_order) for out_vertex in out_link]


23/10/15 19:58:50 WARN StandaloneAppClient$ClientEndpoint: Drop UnregisterApplication(null) because has not yet connected to master


## Question 4: Calculate Ranks of Vertices (30 points)
---
In this question you will be performing an iterative update calculation to determine the rank of each vertex in our dataset.  This is accomplisehd by performing the following:

 1. Initializing an RDD with key/value pairs, where the key is the vertex in our results from Question 2 and the value is the initial rank value of 1.
 2. You then join the rank RDD with our result from Question 2.
 3. The result from Step 2 should then be used to calculate the rank contribution values to pass each vertex using our funciton in Question 3.
 4. You then reduce (sum) all the rank contributions from Step 3 for each vertex and construct a new rank RDD with the rank value being `0.15+0.85*x` where `x` is the sum of all the contributions for a given vertex calculated in Step 3.
 5. Repeat Steps 2 through 4 ten times.  

Congratulations, you have completed the page rank algorithm that Google was first developed on! You are now ready to go start your own competing search engine company.

__NOTE:__ Once you have completed this algorithm, you should persist the rank results and unpersist the results from Question 2 as we are done with them. 

---

In [ ]:
#Place your answer here.

num_iterations = 10

initial_rank = 1.0
ranks = vertex_structure_rdd.keys().map(lambda vertex: (vertex, initial_rank))

def process_vertex_entry(entry):
    _, (links, current_rank) = entry  # Ignore the current vertex ID
    n = len(links)
    redistribution_factor = current_rank / n if n > 0 else 0.0
    result = [(link, redistribution_factor) for link in links]
    return result

for _ in range(num_iterations):
    contribution = vertex_structure_rdd.join(ranks).flatMap(process_vertex_entry)

    ranks = contribution.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank)


ranks.persist()

vertex_structure_rdd.unpersist()


NameError: name 'vertex_structure_rdd' is not defined

## Question 5: Find the Top-k Ranked Vertices (20 points)
---

Using the methods presented in lecture. Use the results of Question 4 to find the Top 10 ranked vertex IDs in the dataset and print out the ID and rank values in descending order of rank.

---

In [ ]:
#Place your answer here.
top_rank = 10

top_rank_vertices = ranks.sortBy(lambda x: x[1], ascending=False).take(top_rank)

for vertex, rank in top_rank_vertices:
    print(f"Vertex ID: {vertex}, Rank: {rank}")


NameError: name 'out_links' is not defined